## 1. 문장의 길이는 문장마다 다르다.

지금까지 Token representation에 대해서 보았습니다. Neural Net 입장에서는 DAG의 첫번째 노드가 지나고나면, sentence는 sequence of vector가 되는 것입니다.

그러면 각 vector들은 각 token이 무슨 의미를 갖고있는지를 갖고있는 것입니다. 그렇다면, 이를 갖고 뭔가 계산을 해서, 벡터 하나가 마지막에 나오고 나면, 그 벡터의 사이즈는 카테고리의 갯수와 같아야 합니다. 그래야만 softmax function에 나오면 distribution과 같이 나올 것입니다.

이 input sentence가 어떤 카테고리에 속할지의 probability를 ouput으로 보여주는 것입니다.
Arbitrary sub-graph가 계산을 하고나면, 마지막에 나온 output이 input의 representation이 되는 것입니다.

이는 사실 카테고리 분포(categorical distribution)으로 muticlass classification한 것과 같습니다.

그런데 sentnece의 size는 고정되어 있지 않습니다. 문장은 긴 문장도 있으며, 짧은 문장도 있습니다. input이 fixed size가 아니라, size가 계속 바뀐다면 어떻게 fixed size representation을 찾을 수 있을까요?

토큰에 대한 의미를 갖고있는 vector를 찾는 것이 table lookup이었다면,
문장에 대한 의미를 갖고있는 vector를 찾는 것이 이 장에서 다룰 문제입니다.

## 2. CBOW(Continuous Bag of words)

토큰 순서가 어떻든, 상관이 없이 bag으로 보겠다는 것입니다. 가방 안의 토큰은 흔들리고 있으며 순서는 상관이 없습니다. 단지, 어떤 단어가 들어있는지만 신경을 쓰게 됩니다.

각 단어, 토큰들은 이미 table lookup을 통해 vector로 바뀐 상태라고 합시다. 이제 그것들의 averaging을 보고자합니다. 각 토큰들이 hi-dimensitional하게 고차원에서 점들로 존재할 것입니다. 그리고 그 점의 average point가 어디있는지를 찾아서 그 포인트가 곧 이 문장을 의미한다고 보는 것입니다.

average를 하는 부분은 DAG에서 그냥 노드 하나입니다. average는 그냥 sum이기 때문에, 자코비안 계산도 간단합니다. 나머지는 무시하고 그냥 input마다 1이 되는 것입니다. Continuous Bag-of-words는 Continuous Bag-of-N-gram으로 generalization도 가능합니다.

토큰 하나만 보면 의미 파악이 부족하다면, 두 토큰씩 묶어서 보려는 시도를 해볼 수 있을 것입니다. 이러한 관점은 named entity나 place name을 파악할 때 중요합니다.

가령, New york university라는 문장이 있다고 합시다. Bag-of-words라고 하면, New와 york과 university를 따로 보고나서 average를 보는 것이었습니다. 하지만 Bag-of-n-gram 관점에서보면, New york을 보고, New york university를 보는 관점으로 훨씬 더 잘 파악할 수 있을 것입니다.

장점은 CBoW는 order(순서)를 무시함에도 성능이 좋다는 점입니다. 다시말해, Text classification과 같은 문제에서 CBoW를 쓰고, 그 뒤에 layer들을 추가하여 classifier를 만들면 성능이 좋은 편입니다.

작년에 build it and break it competition이라는 국제 학술 행사가 있었습니다. sentiment analysis를 하는 data를 주고, classifier하는 문제였습니다. 조경현 교수님은 Bag-of-n-gram classifier로 참여했고, 많은 사람들이 CNN이나 RNN으로 도전했습니다. 그리고 결과는 Bag-of-n-gram classifier의 성능이 굉장히 좋게 나왔습니다.

실제로 practice에서는 Text classification의 base로는 CBoW로 잡는 것이 좋습니다. 게다가 실제로 구현할 필요조차 없습니다. 페이스북 리서치팀에서 FastText라는 것을 개발해놓았기 때문입니다.

## 3. CBoW(Continuous Bag-of-words) Based Multi-Class text Classifier



문장이 있고, 문장을 토큰 단위로 나눕니다.
각 토큰은 x1, x2, ... xt와 같이 표현할 수 있겠습니다.
그리고 각 토큰에 대해서 동일한 weight로 Table Lookup 과정을 거칩니다.
그러면 전부 vector가 될 것입니다.

즉, sequence of token이 sequence of vetor가 되는 것입니다. 그리고 sequence of vetor를 average node를 이용해서 전체 averaging을 합니다. 그리고 이 값은 input sentence에 대해서 sentence representaion이 되는 것입니다.

이 averaging된 결과값은 representation space 안에서 비슷한 의미를 가진 문장들은 가까운 위치에 있으며, 다른 의미를 가진 문장들은 멀리있는 형태가 될 것입니다.

즉, 이는 Text Classification 문제를 풀기에 적합한 상태입니다. 예를 들어서 sentiment analysis를 한다고 하면, 문장에 positive한 sentiment의 단어들을 많이 갖고있는 문장은 서로 가까울 것이며, negative한 sentiment의 단어들을 많이 갖고있는 문장은 서로 가까울 것이기 때문입니다.

그러면 이제 representation이 된 vector를 arbitrary한 sub-graph를 통하고나서, softmax classifier에 넣고나면, 전체에 대한 classifier가 나오는 것입니다.



## 4. 정형화된 Representation 방법은 없다.

결국 sentence represenation은 NN이나 머신러닝 관점에서보면, Universial한 representation(모든 문제를 포용하는 표현 방법)이 있다기 보다는 내가 가진 문제를 푸는 것에 가장 적합한 represenation이 무엇이냐를 의미하는 것입니다.

sentiment analysis를 하는 툴을 만들었다고 하면, representation은 sentiment analysis을 하는데 적합한 representation이어야 합니다.

만약 category classification을 하는 툴을 만들었다고 하면, representation이 category classification에 적합한 representation이어야 합니다.

그리고 이 두 representation은 compatible할 수도 있고, 안 할 수도 있습니다. 다시 말해, 모델을 어떻게 만드냐에 따라서 representation을 따라갑니다.
사실 NN이라는 것은 어느정도 규칙성을 띄고 있습니다. forward-pass computation을 만들고, Probability distribution이 나오게 만들었다면, 처음에 언급한 것처럼 negative log probability를 계산하게 만들고, Backpropgation을 쓰고, SGD를 쓰고, early stoppin을 하면 바로 training을 할 수 있었습니다.

그리고 이러한 정형화된 NN의 매커니즘에 input이 텍스트가 아니라 비디오 프레임을 넣으면 비디오 Classification도 가능하게 됩니다. 물론, CBoW만으로는 원하는 답일 수는 없습니다.

이제 다음 내용에서는 CBoW와는 달리 단어의 순서도 신경스고, 각 단어들의 관계가 어떻게 되는지를 보게 될 것입니다.



## 5. Relational Network

Relation Network(Skip-Bigram): 문장안에 있는 모든 토큰 쌍(pairs)을 보고, 각 쌍에 대해서 신경망을 만들어서 문장표현을 찾습니다.

장점: 여러 단어로 된 표현을 탐지 할 수 있습니다.

단점: 모든 단어간의 관계를 보기 때문에, 전혀 연관이 없는 단어도 보게 됩니다.

## 6. Convolutional Network

Convolution Neural Network(CNN):

특징: k-gram을 계층적으로(hierachically) 보게 됩니다. Layer 를 쌓을 때 마다, 점진 적으로 넓은 범위를 보기 때문에, "단어> 다중 단어 표현> 구절 > 문장"순으로 보는 인간의 인식과도 알맞습니다.  1차원의 Convolutional Network 입니다.

장점: 좁은 지역간 단어의 관계를 볼수 있습니다.

## 7. Self Attention Network

<img src="../images/self-attention.png">

## 8. RNN

<img src="../images/rnn.png">